In [7]:
import pandas as pd
import numpy as np
import json
import requests
import statsmodels.api as sm

In [1]:
import requests
import pandas as pd
import json

headers = {
    'Accept': '*/*',
    'Accept-Language': 'en-US,en;q=0.9',
    'Connection': 'keep-alive',
    'Origin': 'https://www.nba.com',
    'Referer': 'https://www.nba.com/',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'Host': 'stats.nba.com',
    'x-nba-stats-origin': 'stats',
    'x-nba-stats-token': 'true'
}

# Fixed URL with VsDivision='' instead of 'E'
url = 'https://stats.nba.com/stats/leaguedashteamstats?Conference=&DateFrom=&DateTo=&Division=&GameScope=&GameSegment=&Height=&ISTRound=&LastNGames=0&LeagueID=00&Location=&MeasureType=Four%20Factors&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=2024-25&SeasonSegment=&SeasonType=Regular%20Season&ShotClockRange=&StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision='

try:
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        print("Successfully got data!")
        print(json.dumps(data, indent=2)[:500])
    else:
        print(f"Failed to get data: Status code {response.status_code}")
        print("Response:", response.text)
except Exception as e:
    print(f"Error occurred: {e}")

Successfully got data!
{
  "resource": "leaguedashteamstats",
  "parameters": {
    "MeasureType": "Four Factors",
    "PerMode": "PerGame",
    "PlusMinus": "N",
    "PaceAdjust": "N",
    "Rank": "N",
    "LeagueID": "00",
    "Season": "2024-25",
    "SeasonType": "Regular Season",
    "PORound": 0,
    "Outcome": null,
    "Location": null,
    "Month": 0,
    "SeasonSegment": null,
    "DateFrom": null,
    "DateTo": null,
    "OpponentTeamID": 0,
    "VsConference": null,
    "VsDivision": null,
    "TeamID": 0,


In [2]:
# Let's see what's in our data
print("Keys in the response:")
print(data.keys())

# The actual stats are usually in a 'resultSets' key
# Let's check the structure of that
if 'resultSets' in data:
    print("\nStructure of results:")
    print(data['resultSets'][0].keys())  # Usually contains 'headers' and 'rowSet'


Keys in the response:
dict_keys(['resource', 'parameters', 'resultSets'])

Structure of results:
dict_keys(['name', 'headers', 'rowSet'])


In [3]:
# Get the headers and rows from the first result set
headers = data['resultSets'][0]['headers']
rows = data['resultSets'][0]['rowSet']

# Create DataFrame
df = pd.DataFrame(rows, columns=headers)

# Let's look at our data
print("Shape of DataFrame:", df.shape)
print("\nColumns:")
print(df.columns.tolist())
print("\nFirst few rows:")
print(df.head())

Shape of DataFrame: (30, 28)

Columns:
['TEAM_ID', 'TEAM_NAME', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'EFG_PCT', 'FTA_RATE', 'TM_TOV_PCT', 'OREB_PCT', 'OPP_EFG_PCT', 'OPP_FTA_RATE', 'OPP_TOV_PCT', 'OPP_OREB_PCT', 'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'EFG_PCT_RANK', 'FTA_RATE_RANK', 'TM_TOV_PCT_RANK', 'OREB_PCT_RANK', 'OPP_EFG_PCT_RANK', 'OPP_FTA_RATE_RANK', 'OPP_TOV_PCT_RANK', 'OPP_OREB_PCT_RANK']

First few rows:
      TEAM_ID          TEAM_NAME  GP   W   L  W_PCT     MIN  EFG_PCT  \
0  1610612737      Atlanta Hawks  59  27  32  0.458  2852.0    0.534   
1  1610612738     Boston Celtics  59  42  17  0.712  2857.0    0.559   
2  1610612751      Brooklyn Nets  58  21  37  0.362  2799.0    0.518   
3  1610612766  Charlotte Hornets  57  14  43  0.246  2751.0    0.500   
4  1610612741      Chicago Bulls  59  23  36  0.390  2842.0    0.550   

   FTA_RATE  TM_TOV_PCT  ...  W_PCT_RANK  MIN_RANK  EFG_PCT_RANK  \
0     0.263       0.152  ...          20         3            19   

In [4]:
df.head(10)

,TEAM_ID,TEAM_NAME,GP,W,L,W_PCT,MIN,EFG_PCT,FTA_RATE,TM_TOV_PCT,...,W_PCT_RANK,MIN_RANK,EFG_PCT_RANK,FTA_RATE_RANK,TM_TOV_PCT_RANK,OREB_PCT_RANK,OPP_EFG_PCT_RANK,OPP_FTA_RATE_RANK,OPP_TOV_PCT_RANK,OPP_OREB_PCT_RANK
0,1610612737,Atlanta Hawks,59,27,32,0.458,2852.0,0.534,0.263,0.152,...,20,3,19,5,21,15,29,13,4,8
1,1610612738,Boston Celtics,59,42,17,0.712,2857.0,0.559,0.220,0.121,...,3,2,6,28,2,21,2,3,24,6
2,1610612751,Brooklyn Nets,58,21,37,0.362,2799.0,0.518,0.245,0.160,...,24,17,25,17,28,12,26,27,6,21
3,1610612766,Charlotte Hornets,57,14,43,0.246,2751.0,0.500,0.226,0.157,...,28,26,30,27,25,5,19,15,17,17
4,1610612741,Chicago Bulls,59,23,36,0.390,2842.0,0.550,0.207,0.144,...,23,8,12,30,18,27,18,7,29,9
5,1610612739,Cleveland Cavaliers,58,48,10,0.828,2784.0,0.588,0.234,0.131,...,1,24,1,23,3,17,4,11,14,20
6,1610612742,Dallas Mavericks,59,31,28,0.525,2842.0,0.551,0.261,0.142,...,15,8,11,6,16,18,7,12,23,27
7,1610612743,Denver Nuggets,58,38,20,0.655,2804.0,0.575,0.254,0.141,...,4,13,2,9,15,7,13,6,26,14
8,1610612765,Detroit Pistons,59,33,26,0.559,2852.0,0.546,0.234,0.154,...,11,3,13,24,22,8,17,25,11,4
9,1610612744,Golden State Warriors,58,31,27,0.534,2789.0,0.530,0.226,0.138,...,14,21,21,26,9,6,11,17,8,10


In [5]:
#Let's run our data and get a statistical summary. First we have to create a new column and do some math into the new column
def calc_dev():
    df['Shooting Dev'] = df['EFG_PCT']-df['OPP_EFG_PCT']
    df['Rebounding Dev'] = df['OREB_PCT']-df['OPP_OREB_PCT']
    df['Turnover Dev'] = df['TM_TOV_PCT']-df['OPP_TOV_PCT']
    df['Free Throw Dev'] = df['FTA_RATE']-df['OPP_FTA_RATE']
    return df
calc_dev()
df.head(5)

,TEAM_ID,TEAM_NAME,GP,W,L,W_PCT,MIN,EFG_PCT,FTA_RATE,TM_TOV_PCT,...,TM_TOV_PCT_RANK,OREB_PCT_RANK,OPP_EFG_PCT_RANK,OPP_FTA_RATE_RANK,OPP_TOV_PCT_RANK,OPP_OREB_PCT_RANK,Shooting Dev,Rebounding Dev,Turnover Dev,Free Throw Dev
0,1610612737,Atlanta Hawks,59,27,32,0.458,2852.0,0.534,0.263,0.152,...,21,15,29,13,4,8,-0.026,0.017,-0.006,0.016
1,1610612738,Boston Celtics,59,42,17,0.712,2857.0,0.559,0.220,0.121,...,2,21,2,3,24,6,0.038,0.000,-0.013,0.016
2,1610612751,Brooklyn Nets,58,21,37,0.362,2799.0,0.518,0.245,0.160,...,28,12,26,27,6,21,-0.037,-0.004,0.004,-0.031
3,1610612766,Charlotte Hornets,57,14,43,0.246,2751.0,0.500,0.226,0.157,...,25,5,19,15,17,17,-0.045,0.020,0.013,-0.021
4,1610612741,Chicago Bulls,59,23,36,0.390,2842.0,0.550,0.207,0.144,...,18,27,18,7,29,9,0.006,-0.025,0.022,-0.020


In [6]:
#Let's create a function that will allow us to easily extract shooting dev, rebounding dev, turnover dev, and free throw dev for a team and year
def get_team_stats(team):
    team_data = df[df['TEAM_NAME'] == team]
    shooting_dev = team_data['Shooting Dev'].values[0]
    rebounding_dev = team_data['Rebounding Dev'].values[0]
    turnover_dev = team_data['Turnover Dev'].values[0]
    free_throw_dev = team_data['Free Throw Dev'].values[0]
    wins = int(team_data['W'].values[0])
    gp = int(team_data['GP'].values[0])
    
    predicted_win_pct = ( 
        0.5006 + 
        (shooting_dev * 4.6282) +
        (rebounding_dev * 1.5397) +
        (turnover_dev * -3.7446) +
        (free_throw_dev * 0.6154))
    
    predicted_wins = round(predicted_win_pct * gp)

    return {
        'Predicted Wins': predicted_wins,
        'Actual Wins': wins
    }

#Let's test our function
get_team_stats('Denver Nuggets')

{'Predicted Wins': 40, 'Actual Wins': 38}